# AirBNB Price Analyis

1. Webscrape AirBNB page to find absolute location of stay
2. Convert absolute location to address
3. Search for address on Zillow
4. Webscrape Zillow of stay to find Zestimate or actual price
5. Compare AirBNB price with Zillow price, analyze the stay premium

### Imports

In [151]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import requests
from googlesearch import search
import json

### Step 1: Webscraping AirBNB Rental

In [2]:
def getNumbers(w):
    nums = ""
    for char in w:
        if (char.isnumeric()):
            nums += char
    return int(nums)

In [161]:
listing_url = "https://www.airbnb.com/rooms/28341203?adults=1&category_tag=Tag%3A8144&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=flex_destinations_search&check_in=2023-09-17&check_out=2023-09-23&federated_search_id=50f47915-f56e-4298-895e-1f92bc7af554&source_impression_id=p3_1685499677_TDa5wouSsqIiMVph"
options = Options()
#options.add_argument("-headless")
driver = webdriver.Firefox(options=options)
driver.get(listing_url)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

elem = driver.find_element("id", "data-deferred-state").get_attribute("innerHTML")
price, info = None, None
while (not price or not info):
    try:
        price = driver.find_elements("class name", "_tyxjp1")[1].text.strip()
        info = [getNumbers(_) for _ in driver.find_element("class name","lgx66tx").text.split("·")]
    except:
        pass
numGuests = info[0]
numBr = info[1]
numBed = info[2]
numBath =  info[3]
lastChars,lng,lat = "","",""
i = 0
for char in elem:
    if (len(lastChars) == 6):
        lastChars = lastChars[1:6] + char
    else:
        lastChars = lastChars + char
    if(lastChars == "\"lat\":"):
        lat = elem[i+1:i+12]
    elif(lastChars == "\"lng\":"):
        lng = elem[i+1:i+12]
    i+=1
while(not lat[len(lat)-1].isnumeric() or not lng[len(lng)-1].isnumeric()):
    if(lat[len(lat)-1].isnumeric()):
        lng = lng[:len(lng)-1]
    else:
        lat = lat[:len(lat)-1]

print(price, "per night")
print(numBr,("Bedroom," if numBr == 1 else "Bedrooms,"),numBath,("Bathroom" if numBath == 1 else "Bathrooms"))
print(lat,lng,sep = ", ")

$145 per night
2 Bedrooms, 1 Bathroom
43.3323, -76.7289


### Step 2. Getting AirBNB Address

In [130]:
api_token = "AAPK3a1b443d322b4817bcf976c67026e67aY3eBBi031akgsnQKM7K7YG2EfSTflxsLFa-iuS81KbBhqpVKs8PnIpZiiKTTmuRd"
api_url = "https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/reverseGeocode?f=pjson&location=" + lng + "," + lat + "&token=" + api_token 
response = requests.get(api_url)
address_json = response.json()
address = address_json["address"]["Address"]
print(address)

13845 Metzger Rd


### Step 3. Getting Associated Zillow Webpage

In [162]:
results = []
for result in search(address + "realtor.com", num_results = 3):
    results.append(result)
driver.get(results[0])
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
estimate = driver.find_element("class name","Text__StyledText-rui__sc-1j9ntoo-0").text

In [163]:
estimate

''